In [0]:
import json
with open('dataset_43428_1.txt', encoding="utf-8") as json_file:
    dataset = json.load(json_file)

In [0]:
import re 
def preproc(text):
    text = re.sub(" +", " ", text)
    useless_symblos = ["\\n","\",","\\","-","—","\'","\"", "«","»","(", ")","&quot"]
    for sym in useless_symblos:
        text = text.replace(sym," ")
    text = re.sub(" +", " ", text)
    text = text.replace(" .",". ")
    text = text.replace(" ,",", ")
    text = text.replace(" ?","? ")
    text = text.replace(" !","! ")
    text = text.replace(" :",": ")
    text = text.replace(" ;","; ")
    text = re.sub(" +", " ", text)
    text = text.strip()

    return(text)

In [0]:
print('The row without preprocessings looks like: ')
print(dataset[0])

The row without preprocessings looks like: 
В Сербии арестован последний военный преступник
Задержан Горан Хаджич , разыскиваемый Гаагским трибуналом за военные преступления в ходе войны на Балканах .

Как сообщает « Лента.ру » , первые неофициальные сообщения об аресте Горана Хаджича появились ещё 19 июля , однако вскоре были опровергнуты .
В ближайшее время о поимке Хаджича должен официально объявить президент Сербии Борис Тадич .
Об этом сообщает сербский телеканал B92 .

Хорватский политик сербского происхождения Горан Хаджич был руководителем Республики Сербская Краина — непризнанного сербского государства , существовавшего на территории Хорватии в 1992 — 1995 годах .
В 2004 году Международный трибунал по бывшей Югославии в Гааге предъявил Хаджичу обвинения по 14 эпизодам военных преступлений и преступлений против человечности .
В частности , ему вменяется причастность к депортациям и убийствам мирного хорватского населения в 1991 — 1993 годах .

За информацию , способную помочь в

In [0]:
for i in range(len(dataset)):
    dell = preproc(dataset[i]).split('\n')
    clean_sentence = ''
    for word in dell:
        clean_sentence = clean_sentence + word + ' '
    dataset[i] = preproc(clean_sentence)

In [0]:
print('After preprocessing: ')
print(dataset[0])

After preprocessing: 
В Сербии арестован последний военный преступник Задержан Горан Хаджич, разыскиваемый Гаагским трибуналом за военные преступления в ходе войны на Балканах. Как сообщает Лента.ру, первые неофициальные сообщения об аресте Горана Хаджича появились ещё 19 июля, однако вскоре были опровергнуты. В ближайшее время о поимке Хаджича должен официально объявить президент Сербии Борис Тадич. Об этом сообщает сербский телеканал B92. Хорватский политик сербского происхождения Горан Хаджич был руководителем Республики Сербская Краина непризнанного сербского государства, существовавшего на территории Хорватии в 1992 1995 годах. В 2004 году Международный трибунал по бывшей Югославии в Гааге предъявил Хаджичу обвинения по 14 эпизодам военных преступлений и преступлений против человечности. В частности, ему вменяется причастность к депортациям и убийствам мирного хорватского населения в 1991 1993 годах. За информацию, способную помочь в поимке Горана Хаджича, в США была назначена наг

In [0]:
baseline = []
for i in range(len(dataset)):
    baseline.append(dataset[i][:300])

In [0]:
import json
with open('baseline_bert.json', 'w', encoding='utf-8') as f:
    json.dump(baseline, f, ensure_ascii=False, indent=4)

In [0]:

with open('baseline_bert.json', encoding="utf-8") as json_file:
    baseline_file = json.load(json_file)
print(baseline_file[0])
print('len of the row: ', len(baseline_file[0]))
print('len of the file: ', len(baseline_file))

В Сербии арестован последний военный преступник Задержан Горан Хаджич, разыскиваемый Гаагским трибуналом за военные преступления в ходе войны на Балканах. Как сообщает Лента.ру, первые неофициальные сообщения об аресте Горана Хаджича появились ещё 19 июля, однако вскоре были опровергнуты. В ближайше
len of the row:  300
len of the file:  200


In [0]:
!pip install transformers

In [0]:
import torch
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertConfig, BertPreTrainedModel, BertForPreTraining, BertForMaskedLM
import torch.nn as nn
from tqdm import tqdm, tqdm_notebook
import os
import re

RUBERT_PATH = 'ru_conversational_cased_L-12_H-768_A-12_pt'
modelpath = os.path.join(RUBERT_PATH,'pytorch_model.bin')

In [0]:
tokenizer = BertTokenizer.from_pretrained(RUBERT_PATH, do_lower_case=False)
config = BertConfig.from_json_file(os.path.join(RUBERT_PATH,'bert_config.json'))
model = BertForPreTraining.from_pretrained(modelpath, config=config)
model.eval()
from torch import load
di = load(modelpath)

In [0]:
def hw4(text): 
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    segments_ids[0] = 0
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    predictions = model(tokens_tensor, token_type_ids=segments_tensors)
    array_of_thensors = []
    for i in range(len(tokenized_text)):
        array_of_thensors.append((predictions[0][0][i].detach().numpy()))
    mean = np.mean(array_of_thensors, axis = 0)
    return mean

In [0]:
def cos(vector1,vector2):
    return np.dot(np.array(vector1),np.array(vector2))/(((np.sum(vector1**2))**0.5) * ((np.sum(vector2**2))**0.5))

In [0]:
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:

import numpy as np 
from nltk.tokenize import sent_tokenize
import networkx as nx
mean_vectors = []
abstract = []
homework6_bert_matrix = []
l = len(dataset) #number of rows
for ind in range(l):
    text = dataset[ind]
    sentences = sent_tokenize(text)
    
    mean_of_sentence = []
    sent = []
    #mean_of_text = []
    for_mean = []
    for sentence in sentences:
        #print(sentence)
        try: 
            yyy = hw4(sentence)
            sent.append(sentence)
            mean_of_sentence.append([sentence, yyy])
            for_mean.append(yyy)
        except:
            continue 
    sim_mat = np.zeros([len(for_mean), len(for_mean)])
    for k in range(len(for_mean)):
        for j in range(len(for_mean)):
            if k != j:
                sim_mat[k][j] = cos(for_mean[k],for_mean[j])
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sent)), reverse=True)
    text_of_abs = ''
    for j in range(len(ranked_sentences)):
        sentence = ranked_sentences[j][1]
        text_of_abs = text_of_abs + sentence +" "
    token1 = text_of_abs.split(" ")
    referat = ''
    for token in token1: 
        if (len(referat) < 300):
            referat = referat + token + " "
           
    #in some cases len of referat can be longer than 300 because it is compared after concatination
    #so we fave to delete the last word
    if (len(referat) > 300):
        referat1 = '' 
        tokens_del = referat.split(" ")
        #2 because the last token always is empty ('')
        tokens_safe = tokens_del[:(len(tokens_del)-2)]
        for token in tokens_safe:
            referat1 = referat1 + token + " "
    else:
        referat1 = referat
    text_of_abs = preproc(referat1)
    print('text: ', text)
    print('Abstract: ',text_of_abs)
    homework6_bert_matrix.append(text_of_abs)

text:  В Сербии арестован последний военный преступник Задержан Горан Хаджич, разыскиваемый Гаагским трибуналом за военные преступления в ходе войны на Балканах. Как сообщает Лента.ру, первые неофициальные сообщения об аресте Горана Хаджича появились ещё 19 июля, однако вскоре были опровергнуты. В ближайшее время о поимке Хаджича должен официально объявить президент Сербии Борис Тадич. Об этом сообщает сербский телеканал B92. Хорватский политик сербского происхождения Горан Хаджич был руководителем Республики Сербская Краина непризнанного сербского государства, существовавшего на территории Хорватии в 1992 1995 годах. В 2004 году Международный трибунал по бывшей Югославии в Гааге предъявил Хаджичу обвинения по 14 эпизодам военных преступлений и преступлений против человечности. В частности, ему вменяется причастность к депортациям и убийствам мирного хорватского населения в 1991 1993 годах. За информацию, способную помочь в поимке Горана Хаджича, в США была назначена награда в 5 млн до

In [0]:
less_than_300 = 0
more_then_300 = 0
for referat in homework6_bert_matrix:
    if (len(referat)<=300):
        less_than_300 = less_than_300 + 1
    else:
        more_then_300 = more_then_300 + 1
print('Number of rows: ', len(homework6_bert_matrix), ' there are')
print('more_then_300: ', more_then_300, ' cases')
print('less_than_300: ', less_than_300, ' cases')
if (more_then_300 == 0):
    print('Everything is okey')

Number of rows:  200  there are
more_then_300:  0  cases
less_than_300:  200  cases
Everything is okey


In [0]:
import json
with open('homework6_bert_matrix.json', 'w', encoding='utf-8') as f:
    json.dump(homework6_bert_matrix, f, ensure_ascii=False, indent=4)

In [0]:
with open('homework6_bert_matrix.json', encoding="utf-8") as json_file:
    rez = json.load(json_file)

In [0]:
homework6_bert = []
for ind in range(l):
    text = dataset[ind]
    s = re.sub(r'\s+', ' ', text, flags=re.M)    
    sentences = sent_tokenize(text)
    mean_of_sentence = []
    mean_of_text = []
    for_mean = []
    for sentence in sentences:
        try: 
            yyy = hw4(sentence)
            mean_of_sentence.append([sentence, yyy])
            for_mean.append(yyy)
        except:
            continue 
    mean_of_text = np.mean(for_mean, axis = 0)
    cos_dist= [] 
    for k in range(len(mean_of_sentence)):
        cos_dist.append([mean_of_sentence[k][0], cos(mean_of_sentence[k][1],mean_of_text)])
    sort = sorted(cos_dist, key=lambda x: x[1], reverse=True)
    text_of_abs = ''
    for j in range(len(sort)):
        sentence = sort[j][0]
        text_of_abs = text_of_abs + sentence +" "
    token1 = text_of_abs.split(" ")
    referat = ''
    for token in token1: 
        if (len(referat) < 300):
            referat = referat + token + " "
           
   
    if (len(referat) > 300):
        referat1 = '' 
        tokens_del = referat.split(" ")
       
        tokens_safe = tokens_del[:(len(tokens_del)-2)]
        for token in tokens_safe:
            referat1 = referat1 + token + " "
    else:
        referat1 = referat
    text_of_abs = preproc(referat1)
    print('Abstract: ',text_of_abs)
    homework6_bert.append(text_of_abs)

Abstract:  В Сербии арестован последний военный преступник Задержан Горан Хаджич, разыскиваемый Гаагским трибуналом за военные преступления в ходе войны на Балканах. После того как в мае 2011 года был арестован бывший командующий армией Республики Сербской Ратко Младич, Хаджич оставался последним
Abstract:  Мой сын после просмотра пятой части сказал по поводу сеанса волшебного психоанализа, что следующим этапом необходимо было заставить Гарри Поттера перед зеркалом противостоять проникновению в подсознание самому себе, а не только давать копаться у себя в мозгах какому то декаденту, обиженному тем,
Abstract:  Прохоров проинформировал председателя правительства о разработке нового проекта по запуску производства на принципиально новой основе малобюджетного городского автомобиля с использованием передовых технологий, цитирует пресс секретаря премьера Дмитрия Пескова Infox.ru. Авто по карману Михаил
Abstract:  Если так и дальше будет продолжаться, то мы будем констатировать, что выборы в 

In [0]:
import json
with open('homework6_bert.json', 'w', encoding='utf-8') as f:
    json.dump(homework6_bert, f, ensure_ascii=False, indent=4)

In [0]:
with open('homework6_bert.json', encoding="utf-8") as json_file:
    rez1 = json.load(json_file)

In [0]:
!pip install Rouge

In [0]:
from rouge import Rouge
rouge = Rouge()
scores_of_baseline = rouge.get_scores(baseline, dataset, avg=True)
print('Grading based on baseline: ')
print(scores_of_baseline)
print('#######')

Grading based on baseline: 
{'rouge-1': {'f': 0.3248727957128814, 'p': 0.984360161629172, 'r': 0.2247066598568663}, 'rouge-2': {'f': 0.3205214344066571, 'p': 0.9826401794414994, 'r': 0.22149139796283143}, 'rouge-l': {'f': 0.35944364068220586, 'p': 0.9826067519457901, 'r': 0.24968684342985717}}
#######


In [0]:
scores_of_baseline = rouge.get_scores(homework6_bert_matrix, dataset, avg=True)
print('Grading based on simularity matrix: ')
print(scores_of_baseline)
print('#######')

Grading based on simularity matrix: 
{'rouge-1': {'f': 0.3317021972620215, 'p': 0.9997871376811593, 'r': 0.2287423422830373}, 'rouge-2': {'f': 0.3180241395491353, 'p': 0.9729281648797439, 'r': 0.21903752412099148}, 'rouge-l': {'f': 0.37125601886486165, 'p': 0.9997727272727271, 'r': 0.2576105402205396}}
#######


In [30]:
scores_of_baseline = rouge.get_scores(homework6_bert, dataset, avg=True)
print('Grading based on cosine distance between mean vector of the text and mean vectors of sentence: ')
print(scores_of_baseline)
print('#######')

Grading based on cosine distance between mean vector of the text and mean vectors of sentence: 
{'rouge-1': {'f': 0.3321290568087795, 'p': 0.9997871376811593, 'r': 0.2291502619195707}, 'rouge-2': {'f': 0.31772296981513415, 'p': 0.9717153792921771, 'r': 0.2188198019201704}, 'rouge-l': {'f': 0.370379229779726, 'p': 0.9986188811188811, 'r': 0.25665534935035433}}
#######


In [0]:
#calculating line by line
for i in range(len(dataset)):
  print('Rouge score for ', i, ' row')
  print('baseline: ',rouge.get_scores(baseline[i], dataset[i]))
  print('The 1st approach: ',rouge.get_scores(homework6_bert_matrix[i], dataset[i]))
  print('The 2nd approach: ',rouge.get_scores(homework6_bert[i], dataset[i]))


Rouge score for  0  row
baseline:  [{'rouge-1': {'f': 0.3478260840268431, 'p': 0.975609756097561, 'r': 0.21164021164021163}, 'rouge-2': {'f': 0.34210526026469684, 'p': 0.975, 'r': 0.2074468085106383}, 'rouge-l': {'f': 0.4062499967013889, 'p': 0.975, 'r': 0.2565789473684211}}]
The 1st approach:  [{'rouge-1': {'f': 0.3421052603220222, 'p': 1.0, 'r': 0.20634920634920634}, 'rouge-2': {'f': 0.3274336255211842, 'p': 0.9736842105263158, 'r': 0.19680851063829788}, 'rouge-l': {'f': 0.39153438838554355, 'p': 1.0, 'r': 0.24342105263157895}}]
The 2nd approach:  [{'rouge-1': {'f': 0.3421052603220222, 'p': 1.0, 'r': 0.20634920634920634}, 'rouge-2': {'f': 0.3274336255211842, 'p': 0.9736842105263158, 'r': 0.19680851063829788}, 'rouge-l': {'f': 0.39153438838554355, 'p': 1.0, 'r': 0.24342105263157895}}]
Rouge score for  1  row
baseline:  [{'rouge-1': {'f': 0.04779411716887736, 'p': 0.975, 'r': 0.024497487437185928}, 'rouge-2': {'f': 0.04662576640408747, 'p': 0.9743589743589743, 'r': 0.023884349465744813